In [51]:
using LowRankApprox
using Plots
using LinearAlgebra
using Jacobi
using Test
gr()
Plots.GRBackend()

Plots.GRBackend()

In [52]:
function TrilinearMap(coord_E, xhat, yhat, zhat)

    
    m = length(xhat)
    P1 = @. 0.125*(1-xhat)*(1-yhat)*(1-zhat)
    P2 = @. 0.125*(1+xhat)*(1-yhat)*(1-zhat)
    P3 = @. 0.125*(1-xhat)*(1+yhat)*(1-zhat)
    P4 = @. 0.125*(1+xhat)*(1+yhat)*(1-zhat)
    P5 = @. 0.125*(1-xhat)*(1-yhat)*(1+zhat)
    P6 = @. 0.125*(1+xhat)*(1-yhat)*(1+zhat)
    P7 = @. 0.125*(1-xhat)*(1+yhat)*(1+zhat)
    P8 = @. 0.125*(1+xhat)*(1+yhat)*(1+zhat)
    P = [P1 P2 P3 P4 P5 P6 P7 P8]
    P_E = P * coord_E
    # x=F_E(xhat)
    X = P_E
    x = X[:, 1]
    y = X[:, 2]
    z = X[:, 3]
    # derivatives of shape functions with respect to xhat
    dP1_dxhat = @. -0.125*(1-yhat)*(1-zhat)
    dP2_dxhat = -dP1_dxhat
    dP3_dxhat = @. -0.125*(1+yhat)*(1-zhat)
    dP4_dxhat = -dP3_dxhat
    dP5_dxhat = @. -0.125*(1-yhat)*(1+zhat)
    dP6_dxhat = -dP5_dxhat
    dP7_dxhat = @. -0.125*(1+yhat)*(1+zhat)
    dP8_dxhat = -dP7_dxhat

    # derivatives of shape functions with respect to yhat
    dP1_dyhat = @. -0.125*(1-xhat)*(1-zhat)
    dP2_dyhat = @. -0.125*(1+xhat)*(1-zhat)
    dP3_dyhat = -dP1_dyhat
    dP4_dyhat = -dP2_dyhat
    dP5_dyhat = @. -0.125*(1-xhat)*(1+zhat)
    dP6_dyhat = @. -0.125*(1+xhat)*(1+zhat)
    dP7_dyhat = -dP5_dyhat
    dP8_dyhat = -dP6_dyhat

    # derivatives of shape functions with respect to zhat
    dP1_dzhat = @. -0.125*(1-xhat)*(1-yhat)
    dP2_dzhat = @. -0.125*(1+xhat)*(1-yhat)
    dP3_dzhat = @. -0.125*(1-xhat)*(1+yhat)
    dP4_dzhat = @. -0.125*(1+xhat)*(1+yhat)
    dP5_dzhat = -dP1_dzhat
    dP6_dzhat = -dP2_dzhat
    dP7_dzhat = -dP3_dzhat
    dP8_dzhat = -dP4_dzhat
    # gradient of P, [dP/dxhat; dP/dyhat; dP/dzhat]
    GradP = [dP1_dxhat dP2_dxhat dP3_dxhat dP4_dxhat dP5_dxhat dP6_dxhat dP7_dxhat dP8_dxhat;
             dP1_dyhat dP2_dyhat dP3_dyhat dP4_dyhat dP5_dyhat dP6_dyhat dP7_dyhat dP8_dyhat;
             dP1_dzhat dP2_dzhat dP3_dzhat dP4_dzhat dP5_dzhat dP6_dzhat dP7_dzhat dP8_dzhat]

    # JT = [[dx/dxhat, dy/dxhat, dz/dxhat],
    #       [dx/dyhat, dy/dyhat, dz/dyhat],
    #       [dx/dzhat, dy/dzhat, dz/dzhat]] (3m x 3)

    JT = GradP * coord_E
    dxdxhat = JT[1:m,1]
    dydxhat = JT[1:m,2]
    dzdxhat = JT[1:m,3]
    dxdyhat = JT[m+1:2*m,1]
    dydyhat = JT[m+1:2*m,2]
    dzdyhat = JT[m+1:2*m,3]
    dxdzhat = JT[2*m+1:3*m,1]
    dydzhat = JT[2*m+1:3*m,2]
    dzdzhat = JT[2*m+1:3*m,3]
    J_E = @. (dxdxhat*dydyhat*dzdzhat + dxdyhat*dydzhat*dzdzhat + dxdzhat*dydxhat*dzdyhat)-(dxdxhat*dydzhat*dzdyhat + dxdyhat*dydxhat*dzdzhat + dxdzhat*dydyhat*dzdxhat)
    
    DF_E = [dxdxhat dxdyhat dxdzhat;dydxhat dydyhat dydzhat;dzdxhat dzdyhat dzdzhat]
    
    return x, y, z, DF_E, J_E
end

TrilinearMap (generic function with 1 method)

In [53]:
function PrimeBasis(coord_E, xhat, yhat, zhat)

    m = length(xhat)
    # supplements from Wheeler, Xue, & Yotov, 2012
    shat1 = [(xhat .* zhat) (-yhat .* zhat) (zeros(m))]
    shat2 = [(2*xhat .* yhat) (-yhat.^2) (zeros(m))]
    shat3 = [(zeros(m)) (-2*xhat .* zhat) (zeros(m))]  
    shat4 = [(xhat.^2 .* zhat) (-2*xhat .* yhat .*zhat) (zeros(m))]
    shat5 = [(zeros(m)) (xhat .* yhat) (-xhat .* zhat)]
    shat6 = [(zeros(m)) (2*yhat .* zhat) (-zhat.^2)]
    shat7 = [(zeros(m)) (zeros(m)) (-2*xhat .* yhat)]
    shat8 = [(zeros(m)) (xhat .* yhat.^2) (-2*xhat .* yhat .* zhat)]
    shat9 = [(-xhat .* yhat) (zeros(m)) (yhat .* zhat)]
    shat10 = [(-xhat.^2) (zeros(m)) (2*xhat .* zhat)]
    shat11 = [(-2*yhat .* zhat) (zeros(m)) (zeros(m))]
    shat12 = [(-2*xhat .* yhat .* zhat) (zeros(m)) (yhat .* zhat.^2)]
    # (x,y,z) are in E
    x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
    
    # we have 24 basis in 3D space
    P1 = zeros(3,m,24)
    P1[1,:,1] = ones(m)
    P1[1,:,2] = x
    P1[1,:,3] = y
    P1[1,:,4] = z
    P1[2,:,5] = ones(m)
    P1[2,:,6] = x
    P1[2,:,7] = y
    P1[2,:,8] = z
    P1[3,:,9] = ones(m)
    P1[3,:,10] = x
    P1[3,:,11] = y
    P1[3,:,12] = z
    # supplement
    P1[1,:,13] = (DF_E[1:m,1] .* shat1[:,1] + DF_E[1:m,2] .* shat1[:,2] + DF_E[1:m,3] .* shat1[:,3]) ./ J_E
    P1[2,:,13] = (DF_E[m+1:2*m,1] .* shat1[:,1] + DF_E[m+1:2*m,2] .* shat1[:,2] + DF_E[m+1:2*m,3] .* shat1[:,3]) ./ J_E
    P1[3,:,13] = (DF_E[2*m+1:3*m,1] .* shat1[:,1] + DF_E[2*m+1:3*m,2] .* shat1[:,2] + DF_E[2*m+1:3*m,3] .* shat1[:,3]) ./ J_E

    P1[1,:,14] = (DF_E[1:m,1] .* shat2[:,1] + DF_E[1:m,2] .* shat2[:,2] + DF_E[1:m,3] .* shat2[:,3]) ./ J_E
    P1[2,:,14] = (DF_E[m+1:2*m,1] .* shat2[:,1] + DF_E[m+1:2*m,2] .* shat2[:,2] + DF_E[m+1:2*m,3] .* shat2[:,3]) ./ J_E
    P1[3,:,14] = (DF_E[2*m+1:3*m,1] .* shat2[:,1] + DF_E[2*m+1:3*m,2] .* shat2[:,2] + DF_E[2*m+1:3*m,3] .* shat2[:,3]) ./ J_E

    P1[1,:,15] = (DF_E[1:m,1] .* shat3[:,1] + DF_E[1:m,2] .* shat3[:,2] + DF_E[1:m,3] .* shat3[:,3]) ./ J_E
    P1[2,:,15] = (DF_E[m+1:2*m,1] .* shat3[:,1] + DF_E[m+1:2*m,2] .* shat3[:,2] + DF_E[m+1:2*m,3] .* shat3[:,3]) ./ J_E
    P1[3,:,15] = (DF_E[2*m+1:3*m,1] .* shat3[:,1] + DF_E[2*m+1:3*m,2] .* shat3[:,2] + DF_E[2*m+1:3*m,3] .* shat3[:,3]) ./ J_E

    P1[1,:,16] = (DF_E[1:m,1] .* shat4[:,1] + DF_E[1:m,2] .* shat4[:,2] + DF_E[1:m,3] .* shat4[:,3]) ./ J_E
    P1[2,:,16] = (DF_E[m+1:2*m,1] .* shat4[:,1] + DF_E[m+1:2*m,2] .* shat4[:,2] + DF_E[m+1:2*m,3] .* shat4[:,3]) ./ J_E
    P1[3,:,16] = (DF_E[2*m+1:3*m,1] .* shat4[:,1] + DF_E[2*m+1:3*m,2] .* shat4[:,2] + DF_E[2*m+1:3*m,3] .* shat4[:,3]) ./ J_E

    P1[1,:,17] = (DF_E[1:m,1] .* shat5[:,1] + DF_E[1:m,2] .* shat5[:,2] + DF_E[1:m,3] .* shat5[:,3]) ./ J_E
    P1[2,:,17] = (DF_E[m+1:2*m,1] .* shat5[:,1] + DF_E[m+1:2*m,2] .* shat5[:,2] + DF_E[m+1:2*m,3] .* shat5[:,3]) ./ J_E
    P1[3,:,17] = (DF_E[2*m+1:3*m,1] .* shat5[:,1] + DF_E[2*m+1:3*m,2] .* shat5[:,2] + DF_E[2*m+1:3*m,3] .* shat5[:,3]) ./ J_E

    P1[1,:,18] = (DF_E[1:m,1] .* shat6[:,1] + DF_E[1:m,2] .* shat6[:,2] + DF_E[1:m,3] .* shat6[:,3]) ./ J_E
    P1[2,:,18] = (DF_E[m+1:2*m,1] .* shat6[:,1] + DF_E[m+1:2*m,2] .* shat6[:,2] + DF_E[m+1:2*m,3] .* shat6[:,3]) ./ J_E
    P1[3,:,18] = (DF_E[2*m+1:3*m,1] .* shat6[:,1] + DF_E[2*m+1:3*m,2] .* shat6[:,2] + DF_E[2*m+1:3*m,3] .* shat6[:,3]) ./ J_E

    P1[1,:,19] = (DF_E[1:m,1] .* shat7[:,1] + DF_E[1:m,2] .* shat7[:,2] + DF_E[1:m,3] .* shat7[:,3]) ./ J_E
    P1[2,:,19] = (DF_E[m+1:2*m,1] .* shat7[:,1] + DF_E[m+1:2*m,2] .* shat7[:,2] + DF_E[m+1:2*m,3] .* shat7[:,3]) ./ J_E
    P1[3,:,19] = (DF_E[2*m+1:3*m,1] .* shat7[:,1] + DF_E[2*m+1:3*m,2] .* shat7[:,2] + DF_E[2*m+1:3*m,3] .* shat7[:,3]) ./ J_E

    P1[1,:,20] = (DF_E[1:m,1] .* shat8[:,1] + DF_E[1:m,2] .* shat8[:,2] + DF_E[1:m,3] .* shat8[:,3]) ./ J_E
    P1[2,:,20] = (DF_E[m+1:2*m,1] .* shat8[:,1] + DF_E[m+1:2*m,2] .* shat8[:,2] + DF_E[m+1:2*m,3] .* shat8[:,3]) ./ J_E
    P1[3,:,20] = (DF_E[2*m+1:3*m,1] .* shat8[:,1] + DF_E[2*m+1:3*m,2] .* shat8[:,2] + DF_E[2*m+1:3*m,3] .* shat8[:,3]) ./ J_E

    P1[1,:,21] = (DF_E[1:m,1] .* shat9[:,1] + DF_E[1:m,2] .* shat9[:,2] + DF_E[1:m,3] .* shat9[:,3]) ./ J_E
    P1[2,:,21] = (DF_E[m+1:2*m,1] .* shat9[:,1] + DF_E[m+1:2*m,2] .* shat9[:,2] + DF_E[m+1:2*m,3] .* shat9[:,3]) ./ J_E
    P1[3,:,21] = (DF_E[2*m+1:3*m,1] .* shat9[:,1] + DF_E[2*m+1:3*m,2] .* shat9[:,2] + DF_E[2*m+1:3*m,3] .* shat9[:,3]) ./ J_E

    P1[1,:,22] = (DF_E[1:m,1] .* shat10[:,1] + DF_E[1:m,2] .* shat10[:,2] + DF_E[1:m,3] .* shat10[:,3]) ./ J_E
    P1[2,:,22] = (DF_E[m+1:2*m,1] .* shat10[:,1] + DF_E[m+1:2*m,2] .* shat10[:,2] + DF_E[m+1:2*m,3] .* shat10[:,3]) ./ J_E
    P1[3,:,22] = (DF_E[2*m+1:3*m,1] .* shat10[:,1] + DF_E[2*m+1:3*m,2] .* shat10[:,2] + DF_E[2*m+1:3*m,3] .* shat10[:,3]) ./ J_E

    P1[1,:,23] = (DF_E[1:m,1] .* shat11[:,1] + DF_E[1:m,2] .* shat11[:,2] + DF_E[1:m,3] .* shat11[:,3]) ./ J_E
    P1[2,:,23] = (DF_E[m+1:2*m,1] .* shat11[:,1] + DF_E[m+1:2*m,2] .* shat11[:,2] + DF_E[m+1:2*m,3] .* shat11[:,3]) ./ J_E
    P1[3,:,23] = (DF_E[2*m+1:3*m,1] .* shat11[:,1] + DF_E[2*m+1:3*m,2] .* shat11[:,2] + DF_E[2*m+1:3*m,3] .* shat11[:,3]) ./ J_E

    P1[1,:,24] = (DF_E[1:m,1] .* shat12[:,1] + DF_E[1:m,2] .* shat12[:,2] + DF_E[1:m,3] .* shat12[:,3]) ./ J_E
    P1[2,:,24] = (DF_E[m+1:2*m,1] .* shat12[:,1] + DF_E[m+1:2*m,2] .* shat12[:,2] + DF_E[m+1:2*m,3] .* shat12[:,3]) ./ J_E
    P1[3,:,24] = (DF_E[2*m+1:3*m,1] .* shat12[:,1] + DF_E[2*m+1:3*m,2] .* shat12[:,2] + DF_E[2*m+1:3*m,3] .* shat12[:,3]) ./ J_E

    P = zeros(3*m,24)    
    P[1:m,:] = P1[1,:,:]
    P[m+1:2*m,:] = P1[2,:,:]
    P[2*m+1:3*m,:] = P1[3,:,:]
    
    return P
end

PrimeBasis (generic function with 1 method)

In [54]:
function GetNormal(coord_E, xhat, yhat, zhat, edge)

    m = length(xhat)
    if edge == "left"
        x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
        dxdzhat = DF_E[1:m,3]
        dydzhat = DF_E[m+1:2*m,3]
        dzdzhat = DF_E[2*m+1:3*m,3]
        dxdxhat = DF_E[1:m,1]
        dydxhat = DF_E[m+1:2*m,1]
        dzdxhat = DF_E[2*m+1:3*m,1]
        
        n1 = @. dydxhat*dzdzhat - dzdxhat*dydzhat
        n2 = @. dzdxhat*dxdzhat - dxdxhat*dzdzhat
        n3 = @. dxdxhat*dydzhat - dydxhat*dxdzhat
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = [n1 ./ leng; n2 ./ leng; n3 ./ leng] 
    
    elseif edge == "right"
        x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
        dxdzhat = DF_E[1:m,3]
        dydzhat = DF_E[m+1:2*m,3]
        dzdzhat = DF_E[2*m+1:3*m,3]
        dxdxhat = DF_E[1:m,1]
        dydxhat = DF_E[m+1:2*m,1]
        dzdxhat = DF_E[2*m+1:3*m,1]
        
        n1 = @. dzdxhat*dydzhat - dydxhat*dzdzhat
        n2 = @. dxdxhat*dzdzhat - dzdxhat*dxdzhat
        n3 = @. dydxhat*dxdzhat - dxdxhat*dydzhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = [n1 ./ leng; n2 ./ leng; n3 ./ leng]
    elseif edge == "bottom"
        x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
        dxdyhat = DF_E[1:m,2]
        dydyhat = DF_E[m+1:2*m,2]
        dzdyhat = DF_E[2*m+1:3*m,2]
        dxdxhat = DF_E[1:m,1]
        dydxhat = DF_E[m+1:2*m,1]
        dzdxhat = DF_E[2*m+1:3*m,1]
        
        n1 = @. dzdxhat*dydyhat - dydxhat*dzdyhat
        n2 = @. dxdxhat*dzdyhat - dzdxhat*dxdyhat
        n3 = @. dydxhat*dxdyhat - dxdxhat*dydyhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = [n1 ./ leng; n2 ./ leng; n3 ./ leng]
    elseif edge == "top"
        x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
        dxdyhat = DF_E[1:m,2]
        dydyhat = DF_E[m+1:2*m,2]
        dzdyhat = DF_E[2*m+1:3*m,2]
        dxdxhat = DF_E[1:m,1]
        dydxhat = DF_E[m+1:2*m,1]
        dzdxhat = DF_E[2*m+1:3*m,1]
        
        n1 = @. dydxhat*dzdyhat - dzdxhat*dydyhat 
        n2 = @. dzdxhat*dxdyhat - dxdxhat*dzdyhat 
        n3 = @. dxdxhat*dydyhat - dydxhat*dxdyhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = [n1 ./ leng; n2 ./ leng; n3 ./ leng]
    elseif edge == "front"
        x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
        dxdyhat = DF_E[1:m,2]
        dydyhat = DF_E[m+1:2*m,2]
        dzdyhat = DF_E[2*m+1:3*m,2]
        dxdzhat = DF_E[1:m,3]
        dydzhat = DF_E[m+1:2*m,3]
        dzdzhat = DF_E[2*m+1:3*m,3]
        
        n1 = @. dydyhat*dzdzhat - dzdyhat*dydzhat
        n2 = @. dzdyhat*dxdzhat - dxdyhat*dzdzhat
        n3 = @. dxdyhat*dydzhat - dydyhat*dxdzhat
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = [n1 ./ leng; n2 ./ leng; n3 ./ leng]
    elseif edge == "back"
        x, y, z, DF_E, J_E = TrilinearMap(coord_E, xhat, yhat, zhat)
        dxdyhat = DF_E[1:m,2]
        dydyhat = DF_E[m+1:2*m,2]
        dzdyhat = DF_E[2*m+1:3*m,2]
        dxdzhat = DF_E[1:m,3]
        dydzhat = DF_E[m+1:2*m,3]
        dzdzhat = DF_E[2*m+1:3*m,3]
        
        n1 = @. dzdyhat*dydzhat - dydyhat*dzdzhat 
        n2 = @. dxdyhat*dzdzhat - dzdyhat*dxdzhat
        n3 = @. dydyhat*dxdzhat - dxdyhat*dydzhat 
        leng = @. sqrt(n1*n1 + n2*n2 + n3*n3)
        n = [n1 ./ leng; n2 ./ leng; n3 ./ leng] 
    else
        println("Error! Enter the Xhat=[xhat, yhat] on the edge of Ehat")
    
    end

    return n
end

GetNormal (generic function with 1 method)

In [55]:
function VondermondeMat(coord_E)
    """
    Input:
    ------
    coord_E: is the coordinate of vertices of element.
    Note
    3---4
    |   |
    1---2
    Output:
    ------
    VM: the 8x8 vondermonde matrix
    """
    nl = GetNormal(coord_E, [0.], [-1.],[0.],"left")
    nr = GetNormal(coord_E, [0.], [1.],[0.],"right")
    nbt = GetNormal(coord_E, [0.], [0.],[-1.],"bottom")
    nt = GetNormal(coord_E, [0.], [0.],[1.],"top")
    nf = GetNormal(coord_E, [1.], [0.],[0.],"front")
    nbk = GetNormal(coord_E, [-1.], [0.],[0.],"back")
    normals = [nbk nl nbt nf nl nbt nbk nr nbt nf nr nbt nbk nl nt nf nl nt nbk nr nt nf nr nt]
    nd1 = [-1.;-1.;-1.]
    nd2 = [1.;-1.;-1.]
    nd3 = [-1.;1.;-1.]
    nd4 = [1.;1.;-1.]
    nd5 = [-1.;-1.;1.]
    nd6 = [1.;-1.;1.]
    nd7 = [-1.;1.;1.]
    nd8 = [1.;1.;1.]
    nodes = [nd1 nd1 nd1 nd2 nd2 nd2 nd3 nd3 nd3 nd4 nd4 nd4 nd5 nd5 nd5 nd6 nd6 nd6 nd7 nd7 nd7 nd8 nd8 nd8]
    # vondermonde matrix, V_ij = phi_j(x_i).n_i
    VM = zeros(24,24)

    for i=1:24
        for j=1:24
            P = PrimeBasis(coord_E, nodes[1,i], nodes[2,i], nodes[3,i])
            VM[i,j] = dot(P[:,j],normals[:,i])
        end
    end

    return VM
end

VondermondeMat (generic function with 1 method)

In [56]:
function GetHexaNodalBasis(coord_E, xhat, yhat, zhat)
   
    VM = VondermondeMat(coord_E)
    P = PrimeBasis(coord_E, xhat, yhat, zhat)
    invVM = inv(VM)
    Nhat = P * invVM

    return Nhat
end

GetHexaNodalBasis (generic function with 1 method)

In [68]:
@testset "TestNodalBasisUniform" begin

    coord_E = [0. 0. 0.;1. 0. 0.;0. 1. 0.;1. 1. 0.;
               0. 0. 1.;1. 0. 1.;0. 1. 1.;1. 1. 1.]
    nl = GetNormal(coord_E, [0.], [-1.],[0.],"left")
    nr = GetNormal(coord_E, [0.], [1.],[0.],"right")
    nbt = GetNormal(coord_E, [0.], [0.],[-1.],"bottom")
    nt = GetNormal(coord_E, [0.], [0.],[1.],"top")
    nf = GetNormal(coord_E, [1.], [0.],[0.],"front")
    nbk = GetNormal(coord_E, [-1.], [0.],[0.],"back")
    
    err = 1e-14
    #============= Node 1==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [-1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 1.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 2==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [-1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 1.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 3==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 1.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 4==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 1.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 5==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [-1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 1.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 6==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [-1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 1.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 7==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 1.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 8==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 1.;atol=err)
end

Test Summary:         | Pass  Total
TestNodalBasisUniform |  192    192


Test.DefaultTestSet("TestNodalBasisUniform", Any[], 192, false, false)

In [69]:
@testset "TestNodalBasisNonUniform" begin

    coord_E = [0. 0. 0.;1. 0. 0.;0. 1. 0.;1. 1. 0.;
               0.25 0.25 1.;0.75 0.25 1.;0.25 0.75 1.;0.75 0.75 1.]
    nl = GetNormal(coord_E, [0.], [-1.],[0.],"left")
    nr = GetNormal(coord_E, [0.], [1.],[0.],"right")
    nbt = GetNormal(coord_E, [0.], [0.],[-1.],"bottom")
    nt = GetNormal(coord_E, [0.], [0.],[1.],"top")
    nf = GetNormal(coord_E, [1.], [0.],[0.],"front")
    nbk = GetNormal(coord_E, [-1.], [0.],[0.],"back")
    
    err = 1e-14
    #============= Node 1==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [-1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 1.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 2==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [-1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 1.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 3==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 1.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 4==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [1], [-1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 1.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 5==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [-1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 1.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 6==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [-1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 1.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 1.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 7==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[-1], [1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 1.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 1.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 0.;atol=err)
    
    #============= Node 8==============#
    # check node 1
    Nhat = GetHexaNodalBasis(coord_E,[1], [1], [1])
    @test isapprox(dot(Nhat[:,1],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,2],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,3],nbt), 0.;atol=err)
    # check node 2
    @test isapprox(dot(Nhat[:,4],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,5],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,6],nbt), 0.;atol=err)
    # check node 3
    @test isapprox(dot(Nhat[:,7],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,8],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,9],nbt), 0.;atol=err)
    # check node 4
    @test isapprox(dot(Nhat[:,10],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,11],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,12],nbt), 0.;atol=err)
    # check node 5
    @test isapprox(dot(Nhat[:,13],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,14],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,15],nt), 0.;atol=err)
    # check node 6
    @test isapprox(dot(Nhat[:,16],nf), 0.;atol=err)
    @test isapprox(dot(Nhat[:,17],nl), 0.;atol=err)
    @test isapprox(dot(Nhat[:,18],nt), 0.;atol=err)
    # check node 7
    @test isapprox(dot(Nhat[:,19],nbk), 0.;atol=err)
    @test isapprox(dot(Nhat[:,20],nr), 0.;atol=err)
    @test isapprox(dot(Nhat[:,21],nt), 0.;atol=err)
    #check node 8
    @test isapprox(dot(Nhat[:,22],nf), 1.;atol=err)
    @test isapprox(dot(Nhat[:,23],nr), 1.;atol=err)
    @test isapprox(dot(Nhat[:,24],nt), 1.;atol=err)
end

Test Summary:            | Pass  Total
TestNodalBasisNonUniform |  192    192


Test.DefaultTestSet("TestNodalBasisNonUniform", Any[], 192, false, false)